In [ ]:
# prove that a grid is a continer for a block and a block is mappaed one to one to an sm
# prove that a block is chopped up to 32 warp or simd groups and they are local an sm
# prove that a warps is a simd group ie action on one warp thread is executed by all the 32 threads ie they move in a lockstep

In [ ]:
# Proof that a block is mapped to a sm
%%writefile example.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// CUDA kernel function that runs on the GPU
// Each thread computes one element of the result vector
__global__ void vectorAdd(int *a, int *b, int *c, int n) {

    // Calculate the global thread ID
    // blockIdx.x = which block this thread belongs to
    // blockDim.x = number of threads per block
    // threadIdx.x = thread index within its block


    const int idx = blockIdx.x * blockDim.x + threadIdx.x;

    //printf("Thread %2d in block %2d: idx=%2d\n", threadIdx.x, blockIdx.x, idx);

    if (idx < n) { c[idx] = a[idx] + b[idx]; }
}

int main(void) {
    // Vector size
    int n = 32;
    size_t bytes = n * sizeof(int);

    // Host (CPU) pointers
    int *h_a, *h_b, *h_c;

    // Device (GPU) pointers
    int *d_a, *d_b, *d_c;

    // Allocate host memory
    h_a = (int*)malloc(bytes);
    h_b = (int*)malloc(bytes);

    //result from gpu
    h_c = (int*)malloc(bytes);

    // Initialize input vectors
    for (int i = 0; i < n; i++) {
        h_a[i] = i;
        h_b[i] = i;
    }

    // Allocate device memory
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);

    // Copy data from host to device
    cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, bytes, cudaMemcpyHostToDevice);


    int threadsPerBlock = 32; // will be by cuda runtime chopped into warps of 32 threads
    int blocksPerGrid = 1;

    printf("Launching kernel with %d blocks and %d threads per block\n", blocksPerGrid, threadsPerBlock);

    // Launch the kernel on the GPU
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);

    // to block the main thread on the cpu until the kernel is done
    cudaDeviceSynchronize();
    cudaMemcpy(h_c, d_c, bytes, cudaMemcpyDeviceToHost);

    //for (int i = 0; i < n; i++) { printf("i -> %2d,   res -> %2d\n", i, h_c[i]); }

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    // Free host memory
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}

Overwriting example.cu


In [ ]:
!nvcc -arch=sm_75 -g -G example.cu -o example

In [ ]:
# set style enabled off <due to ANSI color issue in collab>
# -g for CPU debug Symbols -G for GPU Debug Symbols
# set cuda break_on_launch all
# info cuda threads
# info cuda warps
# info cuda sms
# cuda thread
# cuda lane
# cuda lane 15
# info locals
# print a[idx]
# print b[idx]
# print c[idx]